In [55]:
#https://rapidapi.com/apidojo/api/yahoo-finance1?endpoint=apiendpoint_f6ed13cb-defc-468f-a6a0-ee4ad28bf8af
import yahoo_fin.stock_info as si
import pandas as pd
import math as math

In [56]:
#Nasdaq: tickers_nasdaq()
#S&P500: tickers_sp500()
#Others: tickers_other()
nasdaq = si.tickers_nasdaq()

In [84]:
import requests
import json
def callApi(url, stock):

    querystring = {"symbol": stock }

    headers = {
        'x-rapidapi-host': "apidojo-yahoo-finance-v1.p.rapidapi.com",
        'x-rapidapi-key': "b057006049msh2dd6f48e0283787p151e0djsnf76cd322583c"
        }
    try:
        response = requests.request("GET", url, headers=headers, params=querystring)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [97]:

def totalDebt(stock, summary_result):  
    if not("financialData" in summary_result.keys()):
        return 0
    if ("totalDebt" in summary_result["financialData"].keys()) and ("raw" in summary_result["financialData"]["totalDebt"].keys()):
        return summary_result["financialData"]["totalDebt"]["raw"]
    else:
        return 0


def priceToBook(stock, summary_result):
    if not("defaultKeyStatistics" in summary_result.keys()):
        return 0
    if ("priceToBook" in summary_result["defaultKeyStatistics"].keys()) and ("raw" in summary_result["defaultKeyStatistics"]["priceToBook"].keys()):
        return summary_result["defaultKeyStatistics"]["priceToBook"]["raw"]
    else:
        return 0

def pegValue(stock, summary_result):
    if not("defaultKeyStatistics" in summary_result.keys()):
        return 0
    if ("pegRatio" in summary_result["defaultKeyStatistics"].keys()) and ("raw" in summary_result["defaultKeyStatistics"]["pegRatio"].keys()):
        return summary_result["defaultKeyStatistics"]["pegRatio"]["raw"]
    else:
        return 0
    
def peValue(stock, summary_result):
    trailingPE = {}
    forwardPE = {}
    if not("summaryDetail" in summary_result.keys()):
        return {"trailingPE": 0.0, "forwardPE":0.0}  
    if ("trailingPE" in summary_result["summaryDetail"].keys()) and ("raw" in summary_result["summaryDetail"]["trailingPE"].keys()):
        trailingPE = summary_result["summaryDetail"]["trailingPE"]["raw"]
    else: 
        trailingPE = 0.0
    if "forwardPE" in summary_result["summaryDetail"].keys() and "raw" in summary_result["summaryDetail"]["forwardPE"].keys():
        forwardPE = summary_result["summaryDetail"]["forwardPE"]["raw"]
    else:
        forwardPE = 0.0
    return {"trailingPE": trailingPE, "forwardPE":forwardPE} 

def currentRatio(stock, summary_result):  
    if not("financialData" in summary_result.keys()):
        return 0
    if ("currentRatio" in summary_result["financialData"].keys()) and ("raw" in summary_result["financialData"]["currentRatio"].keys()):
        return summary_result["financialData"]["currentRatio"]["raw"]
    else:
        return 0

In [76]:
def equity(stock):
    summary_result = callApi("https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-balance-sheet", stock)
    if not("balanceSheetHistoryQuarterly" in summary_result.keys()):
        return 0
    if ("balanceSheetStatements" in summary_result["balanceSheetHistoryQuarterly"].keys()) and  len(summary_result["balanceSheetHistoryQuarterly"]["balanceSheetStatements"]) > 0:
        balanceSheetLastStatements = summary_result["balanceSheetHistoryQuarterly"]["balanceSheetStatements"][0]
        if ("totalStockholderEquity" in balanceSheetLastStatements.keys()) and  ("raw" in balanceSheetLastStatements["totalStockholderEquity"].keys()):
            return balanceSheetLastStatements["totalStockholderEquity"]["raw"]
    else:
        return 0

In [100]:

tickers = ['LUV','TELA', 'FB', 'MAR', 'JPM','HTZ', 'DAL' ]
dataSet = [] 
for ticker in tickers:
    data = {}
    points = 0
    summary_result = callApi("https://apidojo-yahoo-finance-v1.p.rapidapi.com/stock/v2/get-summary", ticker)
    data["Ticker"] = ticker
    #
    # Dept
    #
    
    # Dept to Equity Ratio
    total_debt = totalDebt(ticker, summary_result)
    toatl_equity = equity(ticker)
    debt_equity_ratio = 0.0
    if toatl_equity != 0:
        debt_equity_ratio = total_debt / toatl_equity
    data["DebtToEquity < 0.5"] = debt_equity_ratio
    if debt_equity_ratio < 0.50 and debt_equity_ratio != 0.0 :
        data["Good D\E"] = True
        points += 1
    else: 
        data["Good D\E"] = False
    # currentRatio
    current_Ratio = currentRatio(ticker, summary_result)
    data["Current Ratio > 1.5"] = current_Ratio
    if current_Ratio > 0.50 :
        data["Good CR"] = True
        points += 1
    else: 
        data["Good CR"] = False
    
    #
    #  BASIC Look
    #
    
    #PEG Ratio < 1is undervalued 
    peg_ratio = pegValue(ticker, summary_result) 
    data["PEG Ratio < 1"] = peg_ratio
    if peg_ratio < 1 and peg_ratio != 0:
        data["PEG Ratio"] = True
    else: 
        data["PEG Ratio"] = False
    
    price_Book = priceToBook(ticker, summary_result)
    pe_Value = peValue(ticker, summary_result)
    data["Price_Book < 1.5"] = price_Book
    data["TrailingPE < 15"] = pe_Value["trailingPE"]
   # data["ForwardPE < 15"] = pe_Value["forwardPE"]
    validate = 0.0
    
    if pe_Value['trailingPE'] != 0:
        validate = pe_Value['trailingPE'] * price_Book
        data["Trailing Value < 22.5"] = validate
        if validate < 22.5 and price_Book != 0:
            data["Trailing Look"] = True
        else: 
            data["Trailing Look"] = False 
    else:
        data["Trailing Value < 22.5"] = 0
        data["Trailing Look"] = False
    
    if peg_ratio < 1 and peg_ratio != 0:
        points += 1
    elif validate < 22.5 and price_Book != 0:
        points += 1

#    if pe_Value['forwardPE'] != 0 and price_Book != 0:
#        validate = pe_Value['forwardPE'] * price_Book
#        data["Forward Value < 22.5"] = validate
#        if validate < 22.5:
#            data["Forward Look"] = True
#        else: 
#            data["Forward Look"] = False
#    else:
#        data["Forward Value < 22.5"] = 0
#       data["Forward Look"] = False
    data["Points"] = points
    dataSet.append(data.copy())
pd.DataFrame(dataSet)

,Ticker,DebtToEquity < 0.5,Good D\E,Current Ratio > 1.5,Good CR,PEG Ratio < 1,PEG Ratio,Price_Book < 1.5,TrailingPE < 15,Trailing Value < 22.5,Trailing Look,Points
0,LUV,1.056812,False,1.696,True,0.78,True,2.136001,129.147540,275.859313,False,2
1,TELA,0.000000,False,25.977,True,0.00,False,2.998409,0.000000,0.000000,False,2
2,FB,0.000000,False,0.000,False,0.00,False,0.000000,34.572020,0.000000,False,0
3,MAR,-0.000000,False,0.000,False,0.00,False,0.000000,0.000000,0.000000,False,0
4,JPM,0.000000,False,0.000,False,-12.86,True,1.345814,13.859954,18.652919,True,1
5,HTZ,26.246869,False,1.347,True,0.01,True,0.402347,0.000000,0.000000,False,2
6,DAL,3.956732,False,0.939,True,0.24,True,2.322538,0.000000,0.000000,False,2
